In [3]:
import transformers
import torch
from transformers import WhisperFeatureExtractor, set_seed, AutoModelForAudioClassification, TrainingArguments, Trainer, WhisperConfig, EarlyStoppingCallback
import datasets
from datasets import load_dataset, Dataset, DatasetDict, ClassLabel
from audiomentations import AddGaussianNoise, TimeStretch, PitchShift, Gain
import random
import pandas as pd
from tqdm import tqdm
import numpy as np
import librosa
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import gc
import matplotlib.pyplot as plt
import seaborn as sns
tqdm.pandas()
set_seed(55)
np.random.seed(55)
random.seed(55)

# Read audio

In [4]:
def read_audio(path):
    try:
        audio, sample_rate = librosa.load(path, sr=16000)
        return audio
    except:
        return np.array([])

In [5]:
manifest = pd.read_excel('manifest_balanced.xlsx')
variants = ['Arkhangelskie', 'Novgorodskie', 'Povolzkie', 'Pskovskie', 'Ryazanskie']
manifest = manifest[manifest['Variant'].isin(variants)]
manifest = manifest[['path', 'Variant', 'text', 'informant']]
manifest['array'] = manifest['path'].progress_apply(read_audio)
manifest['sampling_rate'] = 16000

100%|████████████████████████████████████████████████████████████████████████████| 25806/25806 [23:59<00:00, 17.93it/s]


# Split on train, validation and test (speakers fixed)

In [6]:
manifest = manifest[['path', 'Variant', 'array', 'sampling_rate', 'informant']]
train = pd.DataFrame()
valid = pd.DataFrame()
test = pd.DataFrame()
variants = ['Arkhangelskie', 'Novgorodskie', 'Povolzkie', 'Pskovskie', 'Ryazanskie']
for variant in variants:
    speaker_counts = manifest[manifest['Variant']==variant]['informant'].value_counts().to_dict()
    total_records = len(manifest[manifest['Variant']==variant]['path'])
    train_target = 0.7 * total_records
    val_target = 0.165 * total_records
    test_target = total_records - train_target - val_target
    speaker_ids = list(speaker_counts.keys())
    train_speakers, val_speakers, test_speakers = [], [], []
    train_sum = val_sum = test_sum = 0
    for speaker in speaker_ids:
        count = speaker_counts[speaker]
        if train_sum + count <= train_target:
            train_speakers.append(speaker)
            train_sum += count
        elif val_sum + count <= val_target:
            val_speakers.append(speaker)
            val_sum += count
        else:
            test_speakers.append(speaker)
            test_sum += count
    train_df = manifest[(manifest['Variant']==variant)&(manifest['informant'].isin(train_speakers))]
    valid_df = manifest[(manifest['Variant']==variant)&(manifest['informant'].isin(val_speakers))]
    test_df = manifest[(manifest['Variant']==variant)&(manifest['informant'].isin(test_speakers))]
    train = pd.concat([train, train_df])
    valid = pd.concat([valid, valid_df])
    test = pd.concat([test, test_df])
train = train[train['array'].str.len()>0]
valid = valid[valid['array'].str.len()>0]
test = test[test['array'].str.len()>0]
try:
    del manifest
    del train_df
    del valid_df
    del test_df
except:
    pass
gc.collect()

7767

In [9]:
train = train.rename(columns={'Variant': 'label'})
valid = valid.rename(columns={'Variant': 'label'})
test = test.rename(columns={'Variant': 'label'})

In [10]:
unique_labels = ['Arkhangelskie', 'Novgorodskie', 'Povolzkie', 'Pskovskie', 'Ryazanskie']
label2id = {label: index for index, label in enumerate(unique_labels)}
id2label = {index: label for index, label in enumerate(unique_labels)}
print("label2id:", label2id)
print("id2label:", id2label)

label2id: {'Arkhangelskie': 0, 'Novgorodskie': 1, 'Povolzkie': 2, 'Pskovskie': 3, 'Ryazanskie': 4}
id2label: {0: 'Arkhangelskie', 1: 'Novgorodskie', 2: 'Povolzkie', 3: 'Pskovskie', 4: 'Ryazanskie'}


# Define types of augmentations and their parameters

In [11]:
augmentations = [AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=1.0),
                 PitchShift(min_semitones=-1.5, max_semitones=1.5, p=1.0),
                 TimeStretch(min_rate=0.9, max_rate=1.1, p=1.0),
                 Gain(min_gain_in_db=-6, max_gain_in_db=6, p=1.0)]

# Applying augmentations to the training set

In [12]:
def apply_random_augmentations(array, path):
    global augmentations
    random.seed(len(path))
    n_aug = random.choice([0, 1, 2])
    aug_chain = random.sample(augmentations, n_aug)
    for aug in aug_chain:
        array = aug(samples=array, sample_rate=16000)
    return array

In [13]:
train['array'] = train.progress_apply(lambda row: apply_random_augmentations(row['array'], row['path']), axis=1)

100%|████████████████████████████████████████████████████████████████████████████| 17347/17347 [07:57<00:00, 36.31it/s]


# Configure the Dataset

In [15]:
ds = DatasetDict({
    'train': Dataset.from_pandas(train[['array', 'label']]).class_encode_column("label"),
    'test': Dataset.from_pandas(valid[['array', 'label']]).class_encode_column("label"),
    'valid': Dataset.from_pandas(test[['array', 'label']]).class_encode_column("label")})

Casting to class labels:   0%|          | 0/17347 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4194 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/3990 [00:00<?, ? examples/s]

In [16]:
ds['train'].features["label"].str2int('Arkhangelskie')

0

In [17]:
try:
    del train
    del valid
    del test
except:
    pass
gc.collect()

25123

# Extract features from audio

In [18]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

In [19]:
def prepare_dataset(batch):
    batch["input_features"] = feature_extractor(batch["array"], sampling_rate=16000, return_tensors='pt').input_features[0]
    return batch

In [20]:
encoded_audio = ds.map(prepare_dataset, remove_columns="array")

Map:   0%|          | 0/17347 [00:00<?, ? examples/s]

Map:   0%|          | 0/4194 [00:00<?, ? examples/s]

Map:   0%|          | 0/3990 [00:00<?, ? examples/s]

In [21]:
encoded_audio = encoded_audio.remove_columns(['__index_level_0__'])

In [22]:
encoded_audio['train'].features

{'label': ClassLabel(names=['Arkhangelskie', 'Novgorodskie', 'Povolzkie', 'Pskovskie', 'Ryazanskie'], id=None),
 'input_features': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None)}

# Define function for testing

In [23]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

# Upload model

In [24]:
num_labels = len(id2label)
config = WhisperConfig.from_pretrained("openai/whisper-tiny", num_labels=num_labels, label2id=label2id, id2label=id2label)
config.classifier_dropout = 0.3
model = AutoModelForAudioClassification.from_pretrained("openai/whisper-tiny", config=config)
device = "cpu"
model.to(device)

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at openai/whisper-tiny and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


WhisperForAudioClassification(
  (encoder): WhisperEncoder(
    (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
    (embed_positions): Embedding(1500, 384)
    (layers): ModuleList(
      (0-3): 4 x WhisperEncoderLayer(
        (self_attn): WhisperSdpaAttention(
          (k_proj): Linear(in_features=384, out_features=384, bias=False)
          (v_proj): Linear(in_features=384, out_features=384, bias=True)
          (q_proj): Linear(in_features=384, out_features=384, bias=True)
          (out_proj): Linear(in_features=384, out_features=384, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (final_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_

# Train classifier only 

In [25]:
def freeze_encoder(model):
    for param in model.encoder.parameters():
        param.requires_grad = False

In [26]:
freeze_encoder(model)

In [27]:
batch_size = 8
training_args_classifier = TrainingArguments(output_dir=f"whisper-tiny-finetuned-rudialect-fixed-speakers-classifier-only-aug",
                                  eval_strategy ="steps",
                                  save_strategy="steps",
                                  learning_rate=1e-4,
                                  eval_steps=200,
                                  save_steps=200,
                                  per_device_train_batch_size=batch_size,
                                  gradient_accumulation_steps=4,
                                  per_device_eval_batch_size=batch_size,
                                  num_train_epochs=5,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  greater_is_better=True,
                                  use_cpu=True, 
                                  report_to="none")

In [28]:
trainer = Trainer(model=model,
                  args=training_args_classifier,
                  train_dataset=encoded_audio["train"].with_format("torch").shuffle(seed=42),
                  eval_dataset=encoded_audio["valid"].with_format("torch"),
                  tokenizer=feature_extractor,
                  compute_metrics=compute_metrics, 
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

C:\Users\Katya\AppData\Local\Temp\ipykernel_8180\185264961.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,


In [29]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
200,No log,1.504384,0.326316,0.180675
400,No log,1.493431,0.374937,0.273403
600,1.469300,1.498460,0.339098,0.279086
800,1.469300,1.482357,0.344862,0.293124
1000,1.288300,1.473760,0.363158,0.305295
1200,1.288300,1.443801,0.369424,0.320977
1400,1.288300,1.443330,0.381203,0.340281
1600,1.167500,1.435370,0.389975,0.354524
1800,1.167500,1.421239,0.397243,0.346710
2000,1.112700,1.406204,0.397744,0.340190


C:\Users\Katya\anaconda3\lib\site-packages\transformers\configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358,

C:\Users\Katya\anaconda3\lib\site-packages\transformers\configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358,

TrainOutput(global_step=2000, training_loss=1.259444061279297, metrics={'train_runtime': 7417.9462, 'train_samples_per_second': 11.693, 'train_steps_per_second': 0.365, 'total_flos': 7.1163353099088e+17, 'train_loss': 1.259444061279297, 'epoch': 3.6841862609497467})

In [30]:
output_test = trainer.predict(encoded_audio["test"].with_format("torch"))

In [31]:
output_test.metrics

{'test_loss': 1.485963225364685,
 'test_accuracy': 0.35216976633285646,
 'test_f1': 0.28890461462661593,
 'test_runtime': 301.4645,
 'test_samples_per_second': 13.912,
 'test_steps_per_second': 1.741}

# Train encoder

In [32]:
def unfreeze_last_n_layers(model, n):
    total_layers = len(model.encoder.layers)
    for i in range(total_layers - n, total_layers):
        for param in model.encoder.layers[i].parameters():
            param.requires_grad = True

In [33]:
unfreeze_last_n_layers(model, 2)

In [34]:
training_args_encoder = TrainingArguments(output_dir=f"whisper-tiny-finetuned-rudialect-fixed-speakers-with-encoder-aug",
                                          eval_strategy="steps",
                                          save_strategy="steps",
                                          eval_steps=300,
                                          save_steps=300,
                                          learning_rate=1e-5,
                                          per_device_train_batch_size=batch_size,
                                          gradient_accumulation_steps=4,
                                          per_device_eval_batch_size=batch_size,
                                          num_train_epochs=5,
                                          load_best_model_at_end=True,
                                          metric_for_best_model="f1",
                                          greater_is_better=True,
                                          use_cpu=True, 
                                          report_to="none")

In [35]:
trainer_encoder = Trainer(model=model,
                          args=training_args_encoder,
                          train_dataset=encoded_audio["train"].with_format("torch").shuffle(seed=42),
                          eval_dataset=encoded_audio["valid"].with_format("torch"),
                          tokenizer=feature_extractor,
                          compute_metrics=compute_metrics, 
                          callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

C:\Users\Katya\AppData\Local\Temp\ipykernel_8180\3655856878.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_encoder = Trainer(model=model,


In [36]:
trainer_encoder.train()

Step,Training Loss,Validation Loss,Accuracy,F1
300,No log,1.200441,0.555138,0.511741
600,0.806600,1.179207,0.563659,0.520825
900,0.806600,1.110458,0.607268,0.561113
1200,0.527300,1.003150,0.637343,0.597491
1500,0.404800,1.045661,0.622306,0.578844
1800,0.404800,0.977330,0.651629,0.612225
2100,0.344200,0.959897,0.658647,0.626309
2400,0.344200,0.933255,0.661654,0.618487
2700,0.308400,0.944918,0.658145,0.621861


C:\Users\Katya\anaconda3\lib\site-packages\transformers\configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358,

C:\Users\Katya\anaconda3\lib\site-packages\transformers\configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358,

TrainOutput(global_step=2700, training_loss=0.4644338113290292, metrics={'train_runtime': 14439.7207, 'train_samples_per_second': 6.007, 'train_steps_per_second': 0.188, 'total_flos': 9.6072141173184e+17, 'train_loss': 0.4644338113290292, 'epoch': 4.9737206085753805})

In [37]:
output_test = trainer_encoder.predict(encoded_audio["test"].with_format("torch"))

In [38]:
output_test.metrics

{'test_loss': 1.280464768409729,
 'test_accuracy': 0.5662851692894612,
 'test_f1': 0.5400392163756708,
 'test_runtime': 301.7255,
 'test_samples_per_second': 13.9,
 'test_steps_per_second': 1.74}

In [39]:
trainer.save_model('/whisper-tiny-finetuned-rudialect-fixed-speakers-with-encoder-aug/model')

C:\Users\Katya\anaconda3\lib\site-packages\transformers\configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358,